In [ ]:
import sys
import pandas as pd
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import awswrangler as wr
from datetime import datetime, date, timedelta
import boto3

# Inicializar o SparkContext e o GlueContext
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Parâmetros do Job
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
job.init(args['JOB_NAME'], args)

In [ ]:
# Definir bucket e prefixo para salvar dados filtrados
caminho_s3 = 'caminho_s3'
nome_tabela = 'nome_tabela'
nome_database = 'nome_database'


# Formata os dias para usar nos selects
hoje = date.today()
semana_passada = hoje - timedelta(days=7)

hoje_formatado = hoje.strftime('%Y%m%d')
semana_passada = semana_passada.strftime('%Y%m%d')
display(hoje_formatado, semana_passada)

In [ ]:
# Pega a base completa antes de semana_passada
select = f"""
SELECT 
    *
FROM NOME_TABELA
WHERE ANOMESDIA <= {semana_passada}
"""

# ESSA FUNÇÃO NÃO PRECISA SER ALTERADA, serve para puxar dados do catalogo de dados do athena
df_base = wr.athena.read_sql_query(
    select
    ,database='workspace_db'
    ,ctas_approach=False
    ,workgroup='workgroup'
)

# Pega a base completa depos de semana passada
select = f"""
SELECT 
    *
FROM NOME_TABELA
WHERE ANOMESDIA >= {semana_passada}
"""

df_novos_registros = wr.athena.read_sql_query(
    select
    ,database='workspace_db'
    ,ctas_approach=False
    ,workgroup='workgroup'
)

In [ ]:
df_base_final = pd.concat([df_base, df_novos_registros])
df = df_base_final


# CONVERTE PARA OS TIPOS DE DADOS DO ATHENA, NÃO É NECESSÁRIO ALTERAR
def obterDados(coluna):
    tipo = coluna.dtypes.name
    if tipo.startswith('int'):
        return 'BIGINT'
    elif tipo.startswith('float'):
        return 'DOUBLE'
    elif tipo.startswith('datetime'):
        return 'TIMESTAMP'
    else:
        return 'STRING'

colunas_e_dados = df.apply(obterDados).to_dict()

#  CRIA A TABELA NO S3/ATHENA, NÃO É NECESSÁRIO ALTERAR
wr.catalog.create_parquet_table(
    database=nome_database
    ,table=nome_tabela
    ,path=caminho_s3
    ,compression='snappy'
    ,columns_types=colunas_e_dados
    ,description='BASE QUE É ATUALIZADA DE HORA EM HORA'
)

# SOBE OS DADOS NESSA TABELA. NÃO É NECESSÁRIO ALTERAR
wr.s3.to_parquet(
    df=df
    ,path=caminho_s3
    ,dataset=True
    ,mode='overwrite'
    ,database=nome_database
    ,table=nome_tabela
)
job.commit()